In [4]:
import os
import sys
import subprocess
import time

# Change to project directory
os.chdir('/scratch/edk202/word2gm-fast/notebooks')
os.chdir("..")

# Clean TensorFlow import with complete silencing
from src.word2gm_fast.utils import (
    import_tensorflow_silently, 
    log_tf_to_file, 
    run_silent_subprocess
)

tf = import_tensorflow_silently()
print(f"✅ TensorFlow {tf.__version__} imported silently")

# Import data pipeline modules
from src.word2gm_fast.dataprep.corpus_to_dataset import make_dataset
from src.word2gm_fast.dataprep.index_vocab import make_vocab
from src.word2gm_fast.dataprep.dataset_to_triplets import build_skipgram_triplets
from src.word2gm_fast.dataprep.tfrecord_io import (
    save_pipeline_artifacts,
    load_pipeline_artifacts,
    write_vocab_to_tfrecord,
    load_vocab_from_tfrecord
)

✅ TensorFlow 2.19.0 imported silently


In [5]:
# Run tests with TensorFlow silencing
result = run_silent_subprocess(
    ['python', '-m', 'unittest', '-b', 'tests.test_corpus_to_dataset'], 
    capture_output=True, text=True
)
if result.returncode == 0:
    print("✅ test_corpus_to_dataset: PASSED")
else:
    print("❌ test_corpus_to_dataset: FAILED")
    print("STDOUT:", result.stdout)
    if result.stderr:
        print("STDERR:", result.stderr)

✅ test_corpus_to_dataset: PASSED


In [6]:
# Run tests with TensorFlow silencing
result = run_silent_subprocess(
    ['python', '-m', 'unittest', '-b', 'tests.test_index_vocab'], 
    capture_output=True, text=True
)
if result.returncode == 0:
    print("✅ test_index_vocab: PASSED")
else:
    print("❌ test_index_vocab: FAILED")
    print("STDOUT:", result.stdout)
    if result.stderr:
        print("STDERR:", result.stderr)

✅ test_index_vocab: PASSED


In [ ]:
# Corpus file information
corpus_file = "1800.txt"
corpus_dir = "/vast/edk202/NLP_corpora/Google_Books/20200217/eng-fiction/5gram_files/6corpus/yearly_files/data"
corpus_path = os.path.join(corpus_dir, corpus_file)
file_size = os.path.getsize(corpus_path) / 1024 / 1024
print("CORPUS FILE: ", corpus_path, "\n")

# Load and filter the corpus
start = time.time()
dataset, _ = make_dataset(corpus_path)
duration_load = time.time() - start
rate_load = file_size / duration_load

# Cache the dataset
dataset = dataset.cache()

# Build the vocabulary from the dataset
start = time.time()
vocab_table = make_vocab(dataset)
duration_vocab = time.time() - start
rate_vocab = file_size / duration_vocab

# Make triplets from the dataset and vocabulary table
start = time.time()
triplets_ds = build_skipgram_triplets(dataset, vocab_table)
duration_triplets = time.time() - start
rate_triplets = file_size / duration_triplets

# Benchmarking output
print("[--    Benchmarks    --]\n")
print(f"{'Step':<35}{'Duration':>10}{'Quantity':>21}{'Rate':>21}")
print("-" * 87)
print(f"{'Corpus loading and filtering (Lazy)':<35}{duration_load:8,.2f}{'s':>2}{file_size:18,.2f}{'MB':>3}{rate_load:16,.2f}{'MB/s':>5}")
print(f"{'Vocabulary creation':<35}{duration_vocab:8,.2f}{'s':>2}{file_size:18,.2f}{'MB':>3}{rate_vocab:16,.2f}{'MB/s':>5}")
print(f"{'Triplet generation (Lazy)':<35}{duration_triplets:8,.2f}{'s':>2}{file_size:18,.2f}{'MB':>3}{rate_triplets:16,.2f}{'MB/s':>5}")

# Create reverse lookup from vocab table once at the beginning
vocab_export = vocab_table.export()
vocab_keys = vocab_export[0].numpy()
vocab_values = vocab_export[1].numpy()
index_to_word = {idx: word.decode('utf-8') for word, idx in zip(vocab_keys, vocab_values)}

# Show sample lines
print("\n[--   Sample Lines   --]\n")
sample_lines = list(dataset.shuffle(1000, seed=42).take(5).as_numpy_iterator())
for line_bytes in sample_lines:
    print(line_bytes.decode("utf-8"))

# Test the vocab table with example words
print("\n[--    Test Words    --]\n")
test_words = ["UNK", "man", "king", "nonexistentword"]
ids = vocab_table.lookup(tf.constant(test_words)).numpy()
print(f"{'Word':<18} {'ID':>6}")
print("-" * 25)
for word, idx in zip(test_words, ids):
    print(f"{word:<18} {idx:>6}")

# Show sample triplets
print("\n[--  Sample Triplets  --]\n")
print(f"{'Center':<8} {'Center Word':<12} {'Positive':<8} {'Pos Word':<12} {'Negative':<8} {'Neg Word':<12}")
print("-" * 75)

sample_triplets = list(triplets_ds.shuffle(1000, seed=123).take(5).as_numpy_iterator())
for triplet in sample_triplets:
    center, positive, negative = triplet
    center_word = index_to_word.get(center, f"ID_{center}")
    pos_word = index_to_word.get(positive, f"ID_{positive}")
    neg_word = index_to_word.get(negative, f"ID_{negative}")
    print(f"{center:<8} {center_word:<12} {positive:<8} {pos_word:<12} {negative:<8} {neg_word:<12}")


CORPUS FILE:  /vast/edk202/NLP_corpora/Google_Books/20200217/eng-fiction/5gram_files/6corpus/yearly_files/data/1800.txt 



RuntimeError: in user code:

    File "/scratch/edk202/word2gm-fast/src/word2gm_fast/dataprep/dataset_to_triplets.py", line 72, in create_triplets  *
        negatives = tf.random.uniform(

    RuntimeError: Random ops require a seed to be set when determinism is enabled. Please set a seed before running the op, e.g. by calling tf.random.set_seed(1).


In [3]:
# Demonstrate TFRecord functionality (optional - commented out for speed)
print("\n[-- TFRecord Demo (Fast Pipeline Serialization) --]\n")

# Example usage (uncomment to actually save/load):
output_dir = "./pipeline_tfrecords"

# Save all pipeline artifacts to TFRecords (one-time cost)
print("Saving pipeline artifacts...")
artifacts = save_pipeline_artifacts(
    dataset, vocab_table, triplets_ds, 
    output_dir=output_dir, 
    compress=True
)
print(f"Saved to: {artifacts['output_dir']}")




[-- TFRecord Demo (Fast Pipeline Serialization) --]

Saving pipeline artifacts...
Saving pipeline artifacts to: ./pipeline_tfrecords
Writing vocabulary TFRecord to: ./pipeline_tfrecords/vocab.tfrecord.gz
Vocabulary write complete. Words written: 33,668
Write time: 0.34 sec
Writing TFRecord to: ./pipeline_tfrecords/triplets.tfrecord.gz
Vocabulary write complete. Words written: 33,668
Write time: 0.34 sec
Writing TFRecord to: ./pipeline_tfrecords/triplets.tfrecord.gz
Write complete. Triplets written: 3,249,693
Write time: 399.83 sec
All artifacts saved successfully!
Saved to: ./pipeline_tfrecords
Write complete. Triplets written: 3,249,693
Write time: 399.83 sec
All artifacts saved successfully!
Saved to: ./pipeline_tfrecords


2025-06-21 14:36:31.250432: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
